# Prevendo os sobreviventes do Titanic
# Modelo 4

In [90]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [91]:
treino = pd.read_csv(r'C:\Users\Lenovo\Documents\MeusProjetos\Data-Science\Titanic\train.csv')
teste = pd.read_csv(r'C:\Users\Lenovo\Documents\MeusProjetos\Data-Science\Titanic\test.csv')

treino.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [92]:
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Concatenando, transformando e separando treino e teste

In [93]:
#Adicionar coluna com a origem
treino['Origem'] = 'treino'
teste['Origem'] = 'teste'

treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Origem
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,treino
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,treino
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,treino
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,treino
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,treino


In [94]:
# Concatenando os DataFrames
df_concatenado = pd.concat([treino, teste], ignore_index=True)
df_concatenado

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Origem
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,treino
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,treino
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,treino
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,treino
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,treino
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,teste
1305,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,teste
1306,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,teste
1307,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,teste


### Transformações

In [95]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_concatenado['Embarked'] = label_encoder.fit_transform(df_concatenado['Embarked'])
df_concatenado['Sex'] = label_encoder.fit_transform(df_concatenado['Sex'])

df_concatenado.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Origem
0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,2,treino
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0,treino
2,3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,treino
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,2,treino
4,5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,2,treino


In [96]:
print(df_concatenado.isna().sum())

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          0
Origem            0
dtype: int64


In [97]:
df_concatenado['Age'].fillna(np.mean(df_concatenado['Age']), inplace=True)
df_concatenado['Fare'].fillna(np.mean(df_concatenado['Fare']), inplace=True)
df_concatenado['Embarked'].fillna(-1, inplace=True)

### Separando os dataframes

In [98]:
df_concatenado.shape

(1309, 13)

In [99]:
# Separando os DataFrames com base na coluna de origem
treino = df_concatenado[df_concatenado['Origem'] == 'treino'].drop(columns=['Origem'])
teste = df_concatenado[df_concatenado['Origem'] == 'teste'].drop(columns=['Origem', 'Survived'])

treino.shape

(891, 12)

In [100]:
teste.shape

(418, 11)

In [101]:
teste.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
891,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,1
892,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,2
893,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,1
894,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,2
895,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,2


In [102]:
print(teste.isna().sum())
print()
print(treino.isna().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


## Selecionando as features

In [103]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'

In [104]:
from sklearn.model_selection import train_test_split
X_treino, X_valid, y_treino, y_valid = train_test_split(treino[features], treino[target], random_state=10, test_size=0.5)

## Verificando parâmetros para a DecisionTreeClassifier

In [105]:
def get_acc_DecisionTree(Xtrain, Xvalid, ytrain, yvalid, SamplesLeaf=[10], LeafNodes=[4], RandomState=0):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import accuracy_score
    
    maior_acc_samples = 0
    melhor_sample = SamplesLeaf[0]
    for samples in SamplesLeaf:
        modelo = DecisionTreeClassifier(random_state=RandomState, min_samples_leaf=samples)
        modelo.fit(Xtrain, ytrain)
        ypred = modelo.predict(Xvalid)
        accuracy = accuracy_score(ypred, yvalid)
        if accuracy > maior_acc_samples:
            maior_acc_samples = accuracy
            melhor_sample = samples

    maior_acc_nodes = 0
    melhor_nodes = LeafNodes[0]

    for nodes in LeafNodes:
        modelo = DecisionTreeClassifier(random_state=RandomState, max_leaf_nodes=nodes)
        modelo.fit(Xtrain, ytrain)
        ypred = modelo.predict(Xvalid)
        accuracy = accuracy_score(ypred, yvalid)
        if accuracy > maior_acc_nodes:
            maior_acc_nodes = accuracy
            melhor_nodes = nodes

    print(f'Melhor min_samples_leaf: {melhor_sample} com {maior_acc_samples:.4f} de acurácia')
    print(f'Melhor max_leaf_nodes: {melhor_nodes} com {maior_acc_nodes:.4f} de acurácia')


In [106]:
get_acc_DecisionTree(X_treino, X_valid, y_treino, y_valid, SamplesLeaf=list(range(2,20, 2)), LeafNodes=list(range(2, 400, 2)))

Melhor min_samples_leaf: 14 com 0.8251 de acurácia
Melhor max_leaf_nodes: 8 com 0.8318 de acurácia


## Treinando o modelo4 com max_leaf_nodes = 8

In [107]:
X_treino, y_treino = treino[features], treino[target]
X_teste = teste[features]

modelo4 = DecisionTreeClassifier(random_state=0, max_leaf_nodes=8)
modelo4.fit(X_treino, y_treino)
y_pred4 = modelo4.predict(X_teste)

y_pred4

array([0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 1.

In [108]:
#Estava em float
y_pred4 = y_pred4.astype(int)
y_pred4

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## Arquivo de submissão

In [109]:
sub = pd.Series(y_pred4, index=teste['PassengerId'], name='Survived')
sub.shape

(418,)

In [110]:
sub.to_csv('Quarto_modelo.csv', header=True)

# Modelo 5 - Random Forests

In [111]:
X_treino, X_valid, y_treino, y_valid = train_test_split(treino[features], treino[target], random_state=0, test_size=0.5)

modelo5 = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_leaf_nodes=8, random_state=0)
modelo5.fit(X_treino, y_treino)
y_pred = modelo5.predict(X_valid)
acc = accuracy_score(y_valid, y_pred)
acc

0.804932735426009

In [112]:
def get_acc_RandomForest(Xtrain, Xvalid, ytrain, yvalid, SamplesLeaf=[10], LeafNodes=[4], estimators=[100],RandomState=0):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    
    maior_acc_samples = 0
    melhor_sample = SamplesLeaf[0]
    for samples in SamplesLeaf:
        modelo = RandomForestClassifier(random_state=RandomState, min_samples_leaf=samples, n_jobs=-1)
        modelo.fit(Xtrain, ytrain)
        ypred = modelo.predict(Xvalid)
        accuracy = accuracy_score(ypred, yvalid)
        if accuracy > maior_acc_samples:
            maior_acc_samples = accuracy
            melhor_sample = samples

    maior_acc_nodes = 0
    melhor_nodes = LeafNodes[0]

    for nodes in LeafNodes:
        modelo = RandomForestClassifier(random_state=RandomState, max_leaf_nodes=nodes, n_jobs=-1)
        modelo.fit(Xtrain, ytrain)
        ypred = modelo.predict(Xvalid)
        accuracy = accuracy_score(ypred, yvalid)
        if accuracy > maior_acc_nodes:
            maior_acc_nodes = accuracy
            melhor_nodes = nodes

    maior_acc_estimators = 0
    melhor_estimators = estimators[0]

    for est in estimators:
        modelo = RandomForestClassifier(random_state=RandomState, n_estimators=est, n_jobs=-1)
        modelo.fit(Xtrain, ytrain)
        ypred = modelo.predict(Xvalid)
        accuracy = accuracy_score(ypred, yvalid)
        if accuracy > maior_acc_estimators:
            maior_acc_estimators = accuracy
            melhor_estimators = est

    print(f'Melhor min_samples_leaf: {melhor_sample} com {maior_acc_samples:.4f} de acurácia')
    print(f'Melhor max_leaf_nodes: {melhor_nodes} com {maior_acc_nodes:.4f} de acurácia')
    print(f'Melhor n_estimators: {melhor_estimators} com {maior_acc_estimators:.4f} de acurácia')

In [113]:
get_acc_RandomForest(X_treino, X_valid, y_treino, y_valid, SamplesLeaf=list(range(2,20, 2)), LeafNodes=list(range(2, 100, 2)), estimators=list(range(40, 241, 40)))

Melhor min_samples_leaf: 6 com 0.8161 de acurácia
Melhor max_leaf_nodes: 30 com 0.8206 de acurácia
Melhor n_estimators: 40 com 0.8004 de acurácia


In [114]:
#validação 1
modelo5 = RandomForestClassifier(random_state=0, max_leaf_nodes=30, n_estimators=40, n_jobs=-1)
modelo5.fit(X_treino, y_treino)
ypred = modelo5.predict(X_valid)
accuracy = accuracy_score(ypred, y_valid)
accuracy

0.8004484304932735

In [115]:
#validação 2
modelo5 = RandomForestClassifier(random_state=0, max_leaf_nodes=30, n_estimators=150, n_jobs=-1)
modelo5.fit(X_treino, y_treino)
ypred = modelo5.predict(X_valid)
accuracy = accuracy_score(ypred, y_valid)
accuracy

0.8094170403587444

In [116]:
#validação 3
modelo5 = RandomForestClassifier(random_state=0, n_estimators=150, min_samples_leaf=6, n_jobs=-1)
modelo5.fit(X_treino, y_treino)
ypred = modelo5.predict(X_valid)
accuracy = accuracy_score(ypred, y_valid)
accuracy

0.8026905829596412

In [117]:
#validação 4
modelo5 = RandomForestClassifier(random_state=0, max_leaf_nodes=30, n_jobs=-1)
modelo5.fit(X_treino, y_treino)
ypred = modelo5.predict(X_valid)
accuracy = accuracy_score(ypred, y_valid)
accuracy

0.820627802690583

## Treinando o modelo 5 com a validação 4

In [118]:
X_treino, y_treino = treino[features], treino[target]
X_teste = teste[features]

modelo5 = RandomForestClassifier(random_state=0, max_leaf_nodes=30, n_jobs=-1)
modelo5.fit(X_treino, y_treino)
y_pred5 = modelo5.predict(X_teste)

y_pred5 = y_pred5.astype(int)

y_pred5

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## Arquivo de submissão

In [119]:
def get_submission(pred, index='PassengerId', name='Survived', name_file='file.csv'):
    import pandas as pd
    sub = pd.Series(pred, index=teste[index], name=name)
    sub.to_csv(name_file, header=True)

In [120]:
get_submission(y_pred5, name_file='Quinto_modelo.csv')

# Modelo 6 - Regressão Logística

In [121]:
from sklearn.linear_model import LogisticRegression

In [122]:
def get_acc_LogisticRegression(train_df, size_test=0.4, features_list=[], target_name='target' ,iterations=[100, 300, 500, 700, 1000], solvers=['lbfgs', 'newton-cg', 'sag', 'saga']):
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score

    X_train, X_valid, y_train, y_valid = train_test_split(train_df[features_list], train_df[target_name], random_state=42, test_size=size_test)

    maior_acc_iter_solver = 0
    melhor_iteration = iterations[0]
    melhor_solver = solvers[0]
    for it in iterations:
        for solv in solvers:
            model = LogisticRegression(random_state=42, solver=solv, max_iter=it, n_jobs=-1)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_valid)
            acc = accuracy_score(y_valid, y_pred)
            if acc >= maior_acc_iter_solver:
                maior_acc_iter_solver = acc
                melhor_iteration = it
                melhor_solver = solv

    print(f'Melhor iteração: {melhor_iteration}')
    print(f'Melhor solver: {melhor_solver}')
    print(f'Acurácia: {maior_acc_iter_solver}')

### Normalizando os dados

In [123]:
def normalizar_colunas_numericas(df, numeric_cols=[]):
    '''
    df: DataFrame com colunas numéricas
    numeric_cols (list): Lista com nome das colunas numéricas
    '''
    from sklearn.preprocessing import MinMaxScaler
    normalizador = MinMaxScaler()
    for col in numeric_cols:
        df[col] = normalizador.fit_transform(df[[col]])

    return df

In [124]:
treino = normalizar_colunas_numericas(treino, ['Age', 'Fare'])
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",1,0.271174,1,0,A/5 21171,0.014151,NaN,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,0.472229,1,0,PC 17599,0.139136,C85,0
2,3,1.0,3,"Heikkinen, Miss. Laina",0,0.321438,0,0,STON/O2. 3101282,0.015469,NaN,2
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,0.434531,1,0,113803,0.103644,C123,2
4,5,0.0,3,"Allen, Mr. William Henry",1,0.434531,0,0,373450,0.015713,NaN,2


In [125]:
get_acc_LogisticRegression(train_df=treino, features_list=features, target_name=target)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Melhor iteração: 1000
Melhor solver: saga
Acurácia: 0.7899159663865546


In [126]:
get_acc_LogisticRegression(train_df=treino, features_list=features, target_name=target, size_test=0.5)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Melhor iteração: 1000
Melhor solver: saga
Acurácia: 0.7937219730941704


In [127]:
get_acc_LogisticRegression(train_df=treino, features_list=features, target_name=target, size_test=0.3)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Melhor iteração: 1000
Melhor solver: saga
Acurácia: 0.8097014925373134


## Treinando o modelo 6 com max_iter=1000 e solver='saga'

In [128]:
X_treino, y_treino = treino[features], treino[target]
X_teste = teste[features]

modelo6 = LogisticRegression(random_state=42, max_iter=1000, solver='saga', n_jobs=-1)
modelo6.fit(X_treino, y_treino)

y_pred6 = modelo6.predict(X_teste)

y_pred6 = y_pred5.astype(int)

y_pred6

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## Arquivo de submissão 6

In [129]:
get_submission(y_pred6, name_file='Sexto_modelo.csv')

In [130]:
y_pred6 == y_pred5

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [131]:
y_pred4 == y_pred6

array([ True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,